In [1]:
from os import listdir
import xarray
import cftime
import numpy as np

import sys
sys.path.append('/projects/dgs/persad_research/heat_research/aerosol-heatwaves/scripts/')
import paths
import compute_thresholds as ct

ModuleNotFoundError: No module named 'compute_thresholds'

In [3]:
import sys
sys.path.append('/projects/dgs/persad_research/heat_research/testing/')
from analysis import calculate_heatwave_statistics

In [21]:
import xarray

ds = xarray.open_dataset("/projects/dgs/persad_research/heat_research/testing/old_threshold.nc")
ds["tnpct"].values = xarray.open_dataset("/projects/dgs/persad_research/heat_research/testing/new_control.nc")["threshold"].values - 273.15
ds["txpct"].values = xarray.open_dataset("/projects/dgs/persad_research/heat_research/testing/new_control.nc")["threshold"].values - 273.15
ds.to_netcdf("/projects/dgs/persad_research/heat_research/testing/modified_control.nc")

In [22]:
mn_path = "/projects/dgs/persad_research/heat_research/old_data/TREFHTMN/ALL/concatenated/trefhtmin_all_0.nc"
mx_path = "/projects/dgs/persad_research/heat_research/old_data/TREFHTMX/ALL/concatenated/trefhtmax_all_0.nc"
control_path = "/projects/dgs/persad_research/heat_research/testing/modified_control.nc"

calculate_heatwave_statistics("old_algo_hw_metrics.nc", mx_path, mn_path, control_path, "3136")

You didn't specify a land-sea mask. It's faster if you do,so this might take a while.


True

In [ ]:
#get_netcdfs = lambda path_to_directory : [path_to_directory + name for name in listdir(path_to_directory) if ".nc" in name]

path = "/projects/dgs/persad_research/heat_research/data/TREFHTMN/ALL/concatenated/trefhtmin_all_17.nc"
all_trefhtmn = xarray.open_dataset(path)["TREFHTMN"]
control_trefhtmn = xarray.open_dataset(paths.CONTROL_TREFHTMN_NETCDF)["TREFHTMN"]#.sel(time=slice(cftime.DatetimeNoLeap(1920, 1, 1, 0, 0, 0, 0, has_year_zero=True), cftime.DatetimeNoLeap(1950, 1, 1, 0, 0, 0, 0, has_year_zero=True)))

In [ ]:
# control_threshold = ct.compute_threshold(control_trefhtmn)
# control_threshold.to_netcdf("/projects/dgs/persad_research/heat_research/data/debug/control_threshold.nc")

In [1]:
-5 % 40

35

In [ ]:
control_threshold = ct.compute_threshold(all_trefhtmn.sel(time=slice("1920", "1950")))

In [ ]:
str(control_threshold.attrs)

In [ ]:
ds = control_threshold["threshold"]
south_hemisphere = np.ones((int(ds.shape[1]/2), ds.shape[2]))
south_hemisphere.resize((ds.shape[1], ds.shape[2]))

north_hemisphere = 1 - south_hemisphere

(ds.mean(dim="day")*north_hemisphere + ds.mean(dim="day")*south_hemisphere).plot()

In [ ]:
control_threshold

In [ ]:
SHS = (300>=,<=9)
SHW = (120>=,<=270)

In [ ]:
def indicate_hot_days(temp_ds, threshold):
    hot_days = np.zeros(temp_ds.values.shape, dtype=int)
    
    # May–September in the Northern Hemisphere and November–March in the Southern Hemisphere
    south_hemisphere = np.ones((int(temp_ds.shape[1]/2), temp_ds.shape[2]))
    south_hemisphere.resize((temp_ds.shape[1], temp_ds.shape[2]))
    north_hemisphere = 1 - south_hemisphere
    
    for index in range(temp_ds.time.values.size):
        day_number = temp_ds.time.values[index].dayofyr
        if 273 >= day_number >= 121:
            # May–September in the Northern Hemisphere
            hot_days[index] = (temp_ds.values[index] > threshold.values[day_number-1])*north_hemisphere
        elif day_number >= 305 or day_number <= 90:
            # November–March in the Southern Hemisphere
            hot_days[index] = (temp_ds.values[index] > threshold.values[day_number-1])*south_hemisphere
    
    return hot_days
    
def index_heatwaves(timeseries, max_break=1, min_duration=3):
    """Identifies the heatwaves in the timeseries"""
    timeseries = np.pad(timeseries, 1)
    
    diff_indices = np.where(np.diff(timeseries) != 0)[0] + 1

    in_heatwave = False
    current_hw_index = 1

    hw_indices = np.zeros(timeseries.shape, dtype=np.short)

    for i in range(diff_indices.shape[0]-1):
        index = diff_indices[i]
        next_index = diff_indices[i+1]

        if timeseries[index] == 1 and in_heatwave:
            hw_indices[index:next_index] = current_hw_index
        elif timeseries[index] == 0 and in_heatwave and next_index-index <= max_break:
            hw_indices[index:next_index] = current_hw_index
        elif timeseries[index] == 1 and not in_heatwave and next_index-index >= min_duration:
            in_heatwave = True
            hw_indices[index:next_index] = current_hw_index
        elif in_heatwave:
            current_hw_index += 1
            in_heatwave = False
    return hw_indices[1:-1]

hot_days = indicate_hot_days(all_trefhtmn, control_threshold["threshold"])
indexed_heatwaves = np.zeros(hot_days.shape, dtype=np.short)

for i in range(hot_days.shape[1]):
    for j in range(hot_days.shape[2]):
        indexed_heatwaves[:, i, j] = index_heatwaves(hot_days[:, i, j])

In [ ]:
start_index = 0
for index in range(all_trefhtmn.time.shape[0]):
    ts = all_trefhtmn.time.values[index]
    if ts.dayofyr == 1 and not start_set:
        start_index = index
        break

# timeseries = all_trefhtmn.time.values[start_index:end_index]
years = timeseries[-1].year - timeseries[0].year + 1

south_hemisphere = np.ones((int(all_trefhtmn.shape[1]/2), all_trefhtmn.shape[2]), dtype=int)
south_hemisphere.resize((all_trefhtmn.shape[1], all_trefhtmn.shape[2]))
north_hemisphere = 1 - south_hemisphere

hwf = np.zeros((years, indexed_heatwaves.shape[1], indexed_heatwaves.shape[2]), dtype=int)
for index in range(start_index, years):
    north_lower, north_upper = (365*index + 121, 365*index + 273 + 1)
    south_lower, south_upper = (365*index + 305, 365*(index+1) + 90 + 1)
    
    hwf[index] = north_hemisphere*np.sum(indexed_heatwaves[north_lower:north_upper] > 0, axis=0)
    if south_upper < indexed_heatwaves.shape[0]:
        hwf[index] += south_hemisphere*np.sum(indexed_heatwaves[south_lower:south_upper] > 0, axis=0)
    
new_hwf = all_hwf_ds.copy()
new_hwf.values = hwf

In [ ]:
ds = xarray.Dataset(
    data_vars=dict(
        HWF=(["year", "lat", "lon"], hwf),
    ),
    coords=dict(
        lon=(["lon"], all_trefhtmn.lon.values),
        lat=(["lat"], all_trefhtmn.lat.values),
        year=np.arange(all_trefhtmn.time.values[0].year, all_trefhtmn.time.values[-1].year+1),
    ),
    attrs=dict(description=f"90th percentile heatwave metrics."),
    )

In [ ]:
control_threshold.attrs

In [ ]:
np.arange(all_trefhtmn.time.values[0].year, all_trefhtmn.time.values[-1].year+1)

In [ ]:
name = "ALL-12-tn90pct_heatwaves_CESM2_rNone_3136_yearly_summer.nc"
all_hwf_ds = xarray.open_dataset(f"/projects/dgs/persad_research/heat_research/data/heat_output/ALL/tmin/{name}")["HWF_tn90pct"]

all_hwf_ds

In [ ]:
con_comp = xarray.open_dataset("../../data/thresholds/CONTROL/control_threshold.nc")["tnpct"]

In [ ]:
((control_threshold.mean(dim="lat").mean(dim="lon") - 273.15) - con_comp).plot()

In [ ]:
name = "ALL-12-tn90pct_heatwaves_CESM2_rNone_3136_yearly_summer.nc"
all_hwf_ds = xarray.open_dataset(f"/projects/dgs/persad_research/heat_research/old_data/heat_output/ALL/tmin/{name}")["HWF_tn90pct"]

f, ax = plt.subplots(1, 1, figsize=(18, 15), facecolor='w')

new = new_hwf.mean(dim="lat").mean(dim="lon").values[:-1]
old = all_hwf_ds.mean(dim="lat").mean(dim="lon").values.astype("timedelta64[D]").astype(int)[:-1]

ax.plot(np.arange(new.size), new, color="Red", label="New HWF")
ax.plot(np.arange(old.size), old, color="Blue", label="Old HWF")
ax.legend()

In [ ]:
dd = all_trefhtmn.sel(time=slice("1990", "1990")).values > control_threshold["threshold"].values

summed = np.sum(dd, axis=0)
diff = control_threshold["threshold"].mean(dim="day").copy()
diff.values = summed# - all_hwf_ds.sel(time=1920).dt.days.values
diff.plot()

In [ ]:
# dd = all_trefhtmn.sel(time=slice("1990", "1990")).values > control_threshold["threshold"].values

ds = all_trefhtmn.sel(time=slice("1990", "1990"))

dd = np.zeros(ds.shape)

for index in range(ds.time.size):
    dd[index] = ds.values[index] > control_threshold["threshold"].values[ds.time.values[index].dayofyr-1]

summed = np.sum(dd, axis=0)
diff = control_threshold["threshold"].mean(dim="day").copy()
diff.values = summed# - all_hwf_ds.sel(time=1920).dt.days.values
diff.plot()